In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=208fa444c8f557a8f5ed6ff83cac490f24bc5977f2def47e41aa785b35eae6ec
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# word_card.csv파일의 위치
file_path = "/content/drive/MyDrive/unique_word_cards.csv"
word_df = pd.read_csv(file_path, header=None, names=['word'])

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np

# 한국어 단어 카드
word_cards = word_df['word'].tolist()

# KoBERT 모델 로드
model_id = 'kykim/bert-kor-base'
tokenizer = BertTokenizer.from_pretrained(model_id)
model = TFBertModel.from_pretrained(model_id)

# 사용자로부터 입력 받은 키워드
user_keyword = input("유사한 단어를 찾을 키워드를 입력하세요: ")

# 키워드의 임베딩 생성
user_keyword_embedding = model(tokenizer(user_keyword, return_tensors='tf'))['last_hidden_state'][:, 0, :]

# 단어 카드의 임베딩 생성
word_card_embeddings = np.array([model(tokenizer(card, return_tensors='tf'))['last_hidden_state'][:, 0, :] for card in word_cards])

# 코사인 유사도 계산
cosine_similarities = tf.keras.losses.cosine_similarity(user_keyword_embedding, word_card_embeddings).numpy()

combined_data = list(zip(word_cards, cosine_similarities.tolist()))
# 유사도가 높은 상위 10개 단어 카드 선택
top_k = 200
top_results = sorted(combined_data, key=lambda x: x[1], reverse=True)[:top_k]

# 결과 출력
print(f"\n'{user_keyword}'와(과) 유사한 단어 카드 상위 {top_k}개:")
# for idx in top_results:
#     print(f"{idx[0]} (유사도: {idx[1]:.4f})")

print(top_results)


All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at kykim/bert-kor-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


유사한 단어를 찾을 키워드를 입력하세요: 화장실

'화장실'와(과) 유사한 단어 카드 상위 200개:
[('저는교회에다녀요(제-종교는-기독교입니다)', [-0.20782628655433655]), ('여성장애인을위한가사도우미를지원받을수있나요', [-0.21527297794818878]), ('저는이도구(기계)를사용해서얘기해요', [-0.2172640860080719]), ('장애인고용지원금융자조건은어떻습니까', [-0.22144582867622375]), ('국민연금의장애연금지급신청하러왔는데요', [-0.22426185011863708]), ('찬양해요(찬송가를-불러요)', [-0.22469888627529144]), ('이해가안되면말하기화면을읽어주세요', [-0.2269541323184967]), ('키를차량내부에놓아둔채로문을잠갔습니다', [-0.22719821333885193]), ('등록금납입증명서발급해주세요', [-0.22810886800289154]), ('뺑뺑이돌려주세요(타요)', [-0.2292417585849762]), ('제고향은00입니다-00에가보셨어요', [-0.23606324195861816]), ('나는말하는-것은-힘들지만당신의말을이해할수있어요', [-0.23877260088920593]), ('AS센터에갈수없으니기사를보내주세요', [-0.23983728885650635]), ('장애우권익문제연구소인권센터에전화걸어주세요', [-0.24115639925003052]), ('싱크로나이즈드-스위밍(수영종목)', [-0.243303582072258]), ('부처님-오신-날에-절에-가요', [-0.24584656953811646]), ('그-가수는-노래를-잘-불러요', [-0.24830226600170135]), ('휠체어에앉아먹을수있는테이블이있나요', [-0.24910803139209747]), ('예아니오질문을하시면저는좀더빠르게대답할수있습니다편하실때다시전화주세요', [-0.25096631050109863]), ('나는말하는-것은-힘들지만_당신

## 단어카드 데이터 전처리 (띄어쓰기)

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

import vertexai
from vertexai.language_models import ChatModel, InputOutputTextPair
from vertexai.preview.language_models import TextGenerationModel

def predict_large_language_model_sample(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    content: str,
    location: str = "us-central1",
    tuned_model_name: str = "",
    ) :
    """Predict using a Large Language Model."""
    vertexai.init(project=project_id, location=location)
    model = ChatModel.from_pretrained(model_name)

    parameters = {
            "temperature": 0.2,  # Temperature controls the degree of randomness in token selection.
            "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
            "top_p": 0.95,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
            "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
        }

    chat =  model.start_chat(
        context="내가 띄어쓰기가 없는 10개의 문장을 list형태로 넘겨주면 너는 각각의 문장에 올바르게 띄어쓰기를 한 문장을 만들고 그걸 10개의 list형태로 반환해줘야해 그리고 반환할때 불용어는 제거해줘",
        examples=[
            InputOutputTextPair(
                input_text="['워드 카드', '반대해요', '다이아몬드', '현장학습', '그대로-주세요', '목사', '휴게실이있습니까', '날씬하다', '자몽', '77(사이즈)', '앵두나무', '사원연수', '영등포지하상가', '죽고싶다', '흑기사', '된장', '소녀시대', '차장', '영화표(티켓)', '사고']",
                output_text="['워드 카드', '반대해요', '다이아몬드', '현장 학습', '그대로 주세요', '목사', '휴게실이 있습니까', '날씬하다', '자몽', '77 사이즈']",
            ),
        ],
    )

    response = chat.send_message(
        content, **parameters
    )
    print(f"Response from Model: {response.text}")

    response = (response.text).replace("'", '')[2:-1].split(',')

    return response
    print(f"Response from Model: {response.text}")
ll = predict_large_language_model_sample('wise-imagery-410607', "chat-bison@002", 0.2, 256, 0.8, 40, str(word_df['word'][:10].tolist()), "us-central1")

Response from Model:  ['워드 카드', '반대해요', '다이아몬드', '현장 학습', '그대로 주세요', '목사', '휴게실이 있습니까', '날씬하다', '자몽', '77 사이즈']


In [ ]:
str(word_df['word'][:20].tolist())

"['워드 카드', '반대해요', '다이아몬드', '현장학습', '그대로-주세요', '목사', '휴게실이있습니까', '날씬하다', '자몽', '77(사이즈)', '앵두나무', '사원연수', '영등포지하상가', '죽고싶다', '흑기사', '된장', '소녀시대', '차장', '영화표(티켓)', '사고']"

In [ ]:
import csv

preprocessing_word_list = []
error_word_list = []
for idx in range(0, len(word_df['word']), 10):
    response = predict_large_language_model_sample('wise-imagery-410607', "chat-bison@002", 0.2, 256, 0.8, 40, str(word_df['word'][idx:idx+10].tolist()), "us-central1")
    if len(response) == 10:
      for res_idx in range(len(response)):
        preprocessing_word_list.append([word_df['word'][idx+res_idx], response[res_idx]])
    else:
      error_word_list.append(response)

with open("preprocessing_word_list.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerows(preprocessing_word_list)

Response from Model:  ['워드 카드', '반대해요', '다이아몬드', '현장 학습', '그대로 주세요', '목사', '휴게실이 있습니까', '날씬하다', '자몽', '77 사이즈']
Response from Model:  ['앵두 나무', '사원 연수', '영등포 지하상가', '죽고 싶다', '흑 기사', '된장', '소녀 시대', '차장', '영화표 티켓', '사고']
Response from Model:  ['면도 해요', '취학 유예했어요', '다음 달 영문', '좋아요', '선풍기', '전', '카세트', '탁재훈', '프로그램', '어서 서둘러요']
Response from Model:  ['생일이 언제예요', '새벽 기도', '점심 시간', '헬로키티', '농촌', '피임', '결혼식 사회 부탁드립니다', '눌러주세요', '해산물', '강풍 주의보']
Response from Model:  ['멋져', '도와주셔서 감사합니다', '닫아요', 'AAC 기기가 부서지다', '월간 도서', '칫솔', '아르바이트', '당기다', '단추를 끼워요', '대리인']
Response from Model:  ['불가사리', '바보', '변화', '복사', '보리', '정수기 어디 있어요', '잘 익혀 먹어요', '저의 말을 계속 무시해요', '일차 함수', '이번주 영문']
Response from Model:  ['녹음기', '다르다', 'IFC 서울', '노선 안내판', '이비인후과', '서울역', '숟가락', '어묵볶음', '불단', '헤드스틱']
Response from Model:  ['일 년에 한 번', '귀중품 맡겨요', '뮤직 비디오', '연료 주입구', '독수리', '스파게티', '설치해요', '법당에 초를 켜요', '고립되다', '하이에나']
Response from Model:  ['코사인', '붙어요', '큰 사이즈로 주세요', '김포 공항', '인천 공항', '성격', '복용하다', '지원 주택', '독특해요', '거문고'

In [ ]:
preprocessing_word_list

[['워드 카드', '워드 카드'],
 ['반대해요', ' 반대해요'],
 ['다이아몬드', ' 다이아몬드'],
 ['현장학습', ' 현장 학습'],
 ['그대로-주세요', ' 그대로 주세요'],
 ['목사', ' 목사'],
 ['휴게실이있습니까', ' 휴게실이 있습니까'],
 ['날씬하다', ' 날씬하다'],
 ['자몽', ' 자몽'],
 ['77(사이즈)', ' 77 사이즈'],
 ['앵두나무', '앵두 나무'],
 ['사원연수', ' 사원 연수'],
 ['영등포지하상가', ' 영등포 지하상가'],
 ['죽고싶다', ' 죽고 싶다'],
 ['흑기사', ' 흑 기사'],
 ['된장', ' 된장'],
 ['소녀시대', ' 소녀 시대'],
 ['차장', ' 차장'],
 ['영화표(티켓)', ' 영화표 티켓'],
 ['사고', ' 사고'],
 ['면도해요', '면도 해요'],
 ['취학유예했어요', ' 취학 유예했어요'],
 ['다음달_영문', ' 다음 달 영문'],
 ['좋아요', ' 좋아요'],
 ['선풍기', ' 선풍기'],
 ['전', ' 전'],
 ['카세트', ' 카세트'],
 ['탁재훈', ' 탁재훈'],
 ['프로그램', ' 프로그램'],
 ['어서-서둘러요', ' 어서 서둘러요']]

## 전처리 data embedding 생성



In [ ]:
import pandas as pd

file_path = "/content/drive/MyDrive/preprocessing_word_list.csv"
word_df = pd.read_csv(file_path, header=None, names=['word'])

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np

# 한국어 단어 카드
word_cards = word_df['word'].tolist()

# KoBERT 모델 로드
model_id = 'kykim/bert-kor-base'
tokenizer = BertTokenizer.from_pretrained(model_id)
model = TFBertModel.from_pretrained(model_id)

# 단어 카드의 임베딩 생성
# word_card_embeddings = np.array([model(tokenizer(card, return_tensors='tf'))['last_hidden_state'][:, 0, :] for card in word_cards])

# 사용자로부터 입력 받은 키워드
user_keyword = input("유사한 단어를 찾을 키워드를 입력하세요: ")

# 키워드의 임베딩 생성
user_keyword_embedding = model(tokenizer(user_keyword, return_tensors='tf'))['last_hidden_state'][:, 0, :]

# 코사인 유사도 계산
cosine_similarities = tf.keras.losses.cosine_similarity(user_keyword_embedding, word_card_embeddings).numpy()

combined_data = list(zip(word_cards, cosine_similarities.tolist()))
# 유사도가 높은 상위 10개 단어 카드 선택
top_k = 200
top_results = sorted(combined_data, key=lambda x: x[1], reverse=False)[:top_k]

# 결과 출력
print(f"\n'{user_keyword}'와(과) 유사한 단어 카드 상위 {top_k}개:")
# for idx in top_results:
#     print(f"{idx[0]} (유사도: {idx[1]:.4f})")

print(top_results)


All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at kykim/bert-kor-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


유사한 단어를 찾을 키워드를 입력하세요: TV 보고싶어요

'TV 보고싶어요'와(과) 유사한 단어 카드 상위 200개:
[(' TV 보고 싶어요', [-0.9666895866394043]), (' 텔레비전 보고 싶어요', [-0.9292294979095459]), (' 영화 보고 싶어요', [-0.8738987445831299]), (' 컴퓨터 하고 싶어요', [-0.8340848684310913]), ('컴퓨터 하고 싶어요', [-0.8340848684310913]), (' 물 먹고 싶어요', [-0.8048751354217529]), (' 자고 싶어요', [-0.7879233956336975]), (' 보고싶다', [-0.7871105074882507]), (' 먹고 싶어요', [-0.7774859666824341]), (' 밖에 나가고 싶어요', [-0.7733954191207886]), (' 외식하고 싶어요', [-0.7720235586166382]), (' 놀고 싶어요', [-0.7716494202613831]), ('밖을 보고 싶어요', [-0.7689002752304077]), (' 운전 면허증 따고 싶어요', [-0.7652944326400757]), (' 쉬고 싶어요', [-0.7647530436515808]), (' 나가고 싶어요', [-0.7642414569854736]), (' 가고 싶어요', [-0.7573987245559692]), (' 머리 하고 싶어요', [-0.7557451725006104]), (' 밖으로 나가고 싶어요', [-0.7535311579704285]), (' 을 보고 싶어요', [-0.7516728639602661]), (' 에어컨 꺼주세요', [-0.7507948279380798]), (' 하기 싫어요', [-0.7504944801330566]), (' 수영 배우고 싶어요', [-0.7495381832122803]), (' 병원에 가고 싶다', [-0.7448067665100098]), (' MP3 듣고 싶어요',

## 단어 data의 embedding값 npy형식으로 저장

In [ ]:
np.save('KAAC_basic_word_card_embed.npy', word_card_embeddings)